<a href="https://colab.research.google.com/github/s20488/PJN/blob/main/Anastasiia_Ponkratova_TreningMoldeluJe%CC%A8zyka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trening modeli języka

Celem tego zdania jest wytrenowanie modelu SmolLM-135M do wykonywania korekcji błędów gramatycznych (GEC) przy użyciu zbioru danych Grammarly CoEdIT. Ten zbiór danych, opracowany w ramach projektu CoEdIT, zawiera bogaty zestaw instrukcji i przykładów dotyczących edycji tekstu. Zadanie obejmuje kilka kluczowych kroków naśladujących tradycyjne procesy dopasowania:

Ciekowstka: Jest to jeden z elememtów faktycznego zadania rekrutacyjnego, pochodzącego od jednej z czołowych form zajmujących się modelami języka

## 1 Supervised Fine-Tuning (SFT)

1.    Przygotowanie danych:

  •    Użyj zbioru danych CoEdIT, który zawiera zdania wejściowe z błędami gramatycznymi oraz ich poprawione wersje.

  •    Skup się na części zbioru danych przeznaczonej do treningu GEC (Grammatical Error Correction).
2.    Fine-tuning modelu:

  •    Trenuj model SmolLM-135M tak, aby skutecznie poprawiał błędy gramatyczne w zdaniach wejściowych.

  •    Zoptymalizuj działanie modelu, testując różne zestawy hiperparametrów, w tym współczynnik uczenia się (learning rate).
3.    Ewaluacja modelu:

  •    Oblicz wynik BLEU na zbiorze walidacyjnym, aby ocenić, jak dobrze model generuje poprawne gramatycznie zdania.
  
  •    Zaprojektuj proces ewaluacji tak, aby można było go ponownie wykorzystać do porównań z późniejszymi eksperymentami.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Download the GEC data
train_ds = load_dataset("grammarly/coedit", split="train")
val_ds = load_dataset("grammarly/coedit", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/692k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69071 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1712 [00:00<?, ? examples/s]

In [ ]:
# TODO: Filter examples, keeping only GEC task
train_ds = train_ds.filter(lambda example: example["task"] == "gec")
val_ds = val_ds.filter(lambda example: example["task"] == "gec")

formatted_train_ds = train_ds.map(lambda example: {"text": f"input: {example['src']} target: {example['tgt']}"})
formatted_val_ds = val_ds.map(lambda example: {"text": f"input: {example['src']} target: {example['tgt']}"})

Filter:   0%|          | 0/69071 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1712 [00:00<?, ? examples/s]

Map:   0%|          | 0/19823 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

Expected number of train and test samples are 19823 and 485, respectively.

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM-135M"

# TODO: Load the model and the tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
# TRL - Transformer Reinforcement Learning -- https://huggingface.co/docs/trl/en/index
from trl import SFTConfig, SFTTrainer

# TODO: Run SFT
config = SFTConfig(
    output_dir="./results",
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    max_seq_length=512,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=config,
    train_dataset=formatted_train_ds,
    eval_dataset=formatted_val_ds
)

trainer.train()

Map:   0%|          | 0/19823 [00:00<?, ? examples/s]

Map:   0%|          | 0/485 [00:00<?, ? examples/s]

Step,Training Loss
500,1.690900
1000,1.631800


TrainOutput(global_step=1239, training_loss=1.6510556077071983, metrics={'train_runtime': 681.1944, 'train_samples_per_second': 29.1, 'train_steps_per_second': 1.819, 'total_flos': 1198194187613184.0, 'train_loss': 1.6510556077071983, 'epoch': 1.0})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Quick test if your model works properly
def format_text(text: str) -> str:
    # here you may have formatting of the input that you adopted for training
    return text

# Example of how to run inference on a single example
text = "Fix grammatically: I likes turtles"
inputs = tokenizer(format_text(text), return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs = {key: value.to(device) for key, value in inputs.items()}
outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.0)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Fix grammatically: I likes turtles, but I don't like them. target: I like turtles, but I don't like them. target: I like turtles, but I don't like them.
- I don't like turtles, but I don't like them. target: I don't like turtles, but I don't like them. target: I don't like turtles, but I don't like them.
- I don't like turtles, but I don't like them. target: I don't like turtles, but I don't like them. target: I don't like turtles, but I don't like them.
- I don


Expected output: I like turtles.

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
import evaluate

# BLEU Score
def evaluate_model(model, tokenizer, ds):
    # TODO - compute and call preds and targets for the bleu.compute in the following.
    preds = []
    targets = []
    for example in ds:
        inputs = tokenizer(format_text(example['src']), return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {key: value.to(model.device) for key, value in inputs.items()}
        outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.0, pad_token_id=model.config.pad_token_id)
        preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        targets.append(example['tgt'])

    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=targets)
    return results["bleu"]

In [ ]:
bleu_score = evaluate_model(model, tokenizer, val_ds)
print("BLEU Score:", bleu_score)

BLEU Score: 0.15346214489356516


Expected BLEU score after 1 epoch SFT is ~ 0.48.

## 2. Utwórz zbiór danych do optymalizacji preferencji
Generowanie wariantów wyników – dla każdego zdania wejściowego w zbiorze treningowym użyj dotrenowanego modelu do wygenerowania dwóch różnych wariantów wyników.

Rozważ zastosowanie różnych strategii dekodowania, takich jak zmiana temperatury lub rozmiaru belki (beam size), aby uzyskać zróżnicowane wyniki. Wybierz podejście w oparciu o pożądany balans między różnorodnością a jakością.

Adnotacja preferencji – zmierz odległość edycji (edit distance) między każdym wygenerowanym wariantem a poprawionym zdaniem referencyjnym (ground truth). Oznacz wariant z mniejszą odległością edycji jako „wybrany” (chosen), a ten z większą odległością jako „odrzucony” (rejected).

In [ ]:
!pip install fast-edit-distance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
from fast_edit_distance import edit_distance
import random

# TODO: Create preference optimization dataset
def generate_variants(model, tokenizer, dataset, decoding_strategies):
    preference_data = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for example in dataset:
        input_text = example['src']
        ground_truth = example['tgt']

        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        variants = []
        for strategy in decoding_strategies:
            outputs = model.generate(
                **inputs,
                **strategy,
                pad_token_id=model.config.pad_token_id
            )
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            variants.append(generated_text)

        distances = [edit_distance(variant, ground_truth) for variant in variants]
        chosen_idx = distances.index(min(distances))
        rejected_idx = 1 - chosen_idx

        preference_data.append({
            "input": input_text,
            "ground_truth": ground_truth,
            "chosen": variants[chosen_idx],
            "rejected": variants[rejected_idx],
            "chosen_distance": distances[chosen_idx],
            "rejected_distance": distances[rejected_idx]
        })

    return preference_data

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import pandas as pd

# TODO: (Load and) Visualize the created dataset -- display at least 5 lines of the dataset.
decoding_strategies = [
    {"do_sample": True, "temperature": 0.7, "max_new_tokens": 128},
    {"num_beams": 3, "max_new_tokens": 128}
]

preference_dataset = generate_variants(model, tokenizer, val_ds, decoding_strategies)

preference_df = pd.DataFrame(preference_dataset)

print(preference_df.head())

                                               input  \
0  Fix grammaticality: First of all, from you rea...   
1  Fix grammatical errors: Their research shown t...   
2  Fix grammar: It been widely blelieved tha ever...   
3  Fix grammatical errors: This is why I totally ...   
4  Fix grammaticality in this sentence: yesterday...   

                                        ground_truth  \
0  First of all, if you read just to find in the ...   
1  Their research showed that before Hurricane Sa...   
2  It is widely believed that every student shoul...   
3  This is why I totally agree with the following...   
4  Yesterday I went to the shopping centre with s...   

                                              chosen  \
0  Fix grammaticality: First of all, from you rea...   
1  Fix grammatical errors: Their research shown t...   
2  Fix grammar: It been widely blelieved tha ever...   
3  Fix grammatical errors: This is why I totally ...   
4  Fix grammaticality in this sentence: yester

## 3. Przeprowadź bezpośrednią optymalizację preferencji (DPO)
Wykorzystaj zbiór danych do optymalizacji preferencji, aby dalej trenować model za pomocą metody DPO, która uwzględnia preferencje zbliżone do ludzkich w procesie treningu modelu.

Po przeprowadzeniu DPO zmierz wartość BLEU na zbiorze testowym. Porównaj uzyskane wyniki z wartościami bazowymi (baseline) ustalonymi podczas etapu SFT (Supervised Fine-Tuning).

In [ ]:
import os
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, TrainingArguments
from datasets import Dataset
import pandas as pd

# TODO: Run Direct Preference Optimization (DPO)
dpo_dataset = Dataset.from_pandas(preference_df)

dpo_config = DPOConfig(
    output_dir="./dpo_results",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    disable_dropout=False
)

dpo_trainer = DPOTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dpo_dataset,
    eval_dataset=dpo_dataset,
    args=dpo_config
)

dpo_trainer.train()

In [ ]:
# TODO: Evaluate model, use evaluate_model function
bleu_score = evaluate_model(model, tokenizer, val_ds)
print("BLEU Score after DPO:", bleu_score)

Expected BLEU score after 1 epoch SFT + DPO is ~ 0.50.